In [1]:
label_dict = {0: '긍정', 1: '부정', 2: '중립'}

In [ ]:
from transformers import TextClassificationPipeline, BertTokenizerFast, TFBertForSequenceClassification, BertTokenizer
import os


# MODEL_NAME = 'fine-tuned-klue-bert-base'
# MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

MODEL_SAVE_PATH = '/home/inter/chat_MBTI/NP_classifier'

# Load Fine-tuning model
# tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    top_k=None
)

In [4]:
max_seq_len = 100
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, padding='max_length')
    # print('input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True) :',input_id)

    padding_count = input_id.count(tokenizer.pad_token_id)
    # print('padding_count = input_id.count(tokenizer.pad_token_id) :',padding_count)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    # print('attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count :',attention_mask)
    token_type_id = [0] * max_seq_len
    # print('token_type_id = [0] * max_seq_len :',token_type_id)

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]
    # print('encoded_input = [input_ids, attention_masks, token_type_ids] :',encoded_input)

    score = np.argmax(model.predict(encoded_input)[0])
    # print('model.predict(encoded_input) :',model.predict(encoded_input))
    # print('model.predict(encoded_input)[0] :',model.predict(encoded_input)[0])
    # print('score = np.argmax(model.predict(encoded_input)[0]) :',score)

    # print('sentence :',new_sentence)
    print(label_dict[score])

In [11]:
while True :
  new_sentence = input('sentence > ')
  if new_sentence == '끝':
    break
  sentiment_predict(new_sentence)

sentence > 좋다
1/1 [==============================] - 19s 19s/step
(단언) 진술하기
sentence > 싫다
1/1 [==============================] - 0s 395ms/step
(지시) 충고/제안하기
sentence > 안녕
1/1 [==============================] - 0s 405ms/step
(단언) 주장하기
sentence > 짜증나
1/1 [==============================] - 0s 398ms/step
(지시) 충고/제안하기
sentence > 끝


In [13]:
import numpy as np

def NP_predict(sentence):

  temp_li = []
  score_li = text_classifier(sentence)[0]
  for i in score_li:
    temp_li.append(i['score'])

  print(label_dict[int(score_li[temp_li.index(max(temp_li))]['label'][6:])])

In [14]:
NP_predict('아낌없이 나눠주세요')

중립
